In [1]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
from PIL import Image
import os

In [2]:
df = pd.read_csv('final_stress_dataset.csv')

In [3]:
df['stress_inverted'] = 8 - df['stress']

In [4]:
df['obstacles_present'] = df['obst'] == 1
df['pavement_bad'] = df['pav'] == 0
df['parking_present'] = df['parked_veh'] == 1
df['no_crossing_present'] = df['cross'] == 0
df['bike_lane_absent'] = df['type3'] == 0
df['road_marking_absent'] = df['marking'] == 0
df['high_speed_present'] = df['type4'] == 0

In [5]:
def map_frequency_to_weight(frequency):
    if frequency == 4:
        return 1.0
    elif frequency in [2, 3]:
        return 0.6
    elif frequency == 1:
        return 0.3
    else:
        return 0.6

def assign_user_weight(row):
    if row['mode'] == 'car':
        return map_frequency_to_weight(row['car_frequency'])
    elif row['mode'] == 'ebike':
        return map_frequency_to_weight(row['bike_frequency'])
    elif row['mode'] == 'escooter':
        return map_frequency_to_weight(row['escooter_frequency'])
    elif row['mode'] == 'walk':
        return 1.0 
    else:
        return 0.6


In [6]:
df['user_weight'] = df.apply(assign_user_weight, axis=1)

In [8]:
stressor_conditions = {
    'obstacles': df['obstacles_present'] & (df['obstacles'] == 1),
    'pavement': df['pavement_bad'] & (df['pavement'] == 1),
    'parking': df['parking_present'] & (df['parking'] == 1),
    'no_crossing': df['no_crossing_present'] & (df['no_crossing'] == 1),
    'bike_lane': df['bike_lane_absent'] & (df['bike_lane'] == 1),
    'road_marking': df['road_marking_absent'] & (df['road_marking'] == 1),
    'speed': df['high_speed_present'] & (df['speed'] == 1),
    'traffic': df['traffic'] == 1
}

In [9]:
summary_tables = []

In [10]:
for stressor, condition in stressor_conditions.items():
    df_filtered = df[condition]
    summary = df_filtered.groupby(['mode', 'scenario']).apply(
        lambda g: (g['stress_inverted'] * g['user_weight']).sum() / g['user_weight'].sum()
    ).reset_index(name=f'weighted_avg_stress_{stressor}')
    summary_tables.append(summary)


C:\Users\Hasan Can Buldu\AppData\Local\Temp\ipykernel_13476\3319637929.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = df_filtered.groupby(['mode', 'scenario']).apply(
C:\Users\Hasan Can Buldu\AppData\Local\Temp\ipykernel_13476\3319637929.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  summary = df_filtered.groupby(['mode', 'scenario']).apply(
C:\Users\Hasan Can Buldu\AppData\Local\Temp\ip

In [11]:
from functools import reduce

final_summary = reduce(
    lambda left, right: pd.merge(left, right, on=['mode', 'scenario'], how='outer'),
    summary_tables
)

In [12]:
final_summary

,mode,scenario,weighted_avg_stress_obstacles,weighted_avg_stress_pavement,weighted_avg_stress_parking,weighted_avg_stress_no_crossing,weighted_avg_stress_bike_lane,weighted_avg_stress_road_marking,weighted_avg_stress_speed,weighted_avg_stress_traffic
0,car,scenario11,NaN,NaN,5.080000,4.500000,4.739130,4.408163,7.000000,4.932432
1,car,scenario16,3.279070,3.360000,NaN,NaN,3.176471,NaN,NaN,2.911765
2,car,scenario17,NaN,NaN,NaN,NaN,3.660000,NaN,NaN,3.471429
3,car,scenario3,3.292308,2.774194,NaN,NaN,NaN,NaN,3.909091,2.981818
4,car,scenario31,4.454545,NaN,NaN,4.400000,4.361702,4.375000,NaN,3.333333
5,car,scenario36,NaN,NaN,NaN,NaN,NaN,NaN,2.545455,2.800000
6,ebike,scenario11,NaN,NaN,5.000000,3.200000,4.600000,4.583333,5.285714,5.357143
7,ebike,scenario16,4.857143,5.500000,NaN,NaN,4.041667,NaN,NaN,5.000000
8,ebike,scenario17,NaN,NaN,NaN,NaN,4.347826,NaN,NaN,3.750000
9,ebike,scenario3,3.666667,4.018519,NaN,NaN,NaN,NaN,4.583333,3.952381


In [13]:
def classify_weighted_stress(score):
    if pd.isna(score):
        return None
    elif score <= 2.2:
        return 'LTS 1'
    elif score <= 3.4:
        return 'LTS 2'
    elif score <= 4.6:
        return 'LTS 3'
    elif score <= 5.8:
        return 'LTS 4'
    else:
        return 'LTS 5'

In [14]:
for col in final_summary.columns:
    if 'weighted_avg_stress' in col:
        class_col = col.replace('weighted_avg_stress_', 'stress_class_')
        final_summary[class_col] = final_summary[col].apply(classify_weighted_stress)


In [15]:
final_summary

,mode,scenario,weighted_avg_stress_obstacles,weighted_avg_stress_pavement,weighted_avg_stress_parking,weighted_avg_stress_no_crossing,weighted_avg_stress_bike_lane,weighted_avg_stress_road_marking,weighted_avg_stress_speed,weighted_avg_stress_traffic,stress_class_obstacles,stress_class_pavement,stress_class_parking,stress_class_no_crossing,stress_class_bike_lane,stress_class_road_marking,stress_class_speed,stress_class_traffic
0,car,scenario11,NaN,NaN,5.080000,4.500000,4.739130,4.408163,7.000000,4.932432,None,None,LTS 4,LTS 3,LTS 4,LTS 3,LTS 5,LTS 4
1,car,scenario16,3.279070,3.360000,NaN,NaN,3.176471,NaN,NaN,2.911765,LTS 2,LTS 2,None,None,LTS 2,None,None,LTS 2
2,car,scenario17,NaN,NaN,NaN,NaN,3.660000,NaN,NaN,3.471429,None,None,None,None,LTS 3,None,None,LTS 3
3,car,scenario3,3.292308,2.774194,NaN,NaN,NaN,NaN,3.909091,2.981818,LTS 2,LTS 2,None,None,None,None,LTS 3,LTS 2
4,car,scenario31,4.454545,NaN,NaN,4.400000,4.361702,4.375000,NaN,3.333333,LTS 3,None,None,LTS 3,LTS 3,LTS 3,None,LTS 2
5,car,scenario36,NaN,NaN,NaN,NaN,NaN,NaN,2.545455,2.800000,None,None,None,None,None,None,LTS 2,LTS 2
6,ebike,scenario11,NaN,NaN,5.000000,3.200000,4.600000,4.583333,5.285714,5.357143,None,None,LTS 4,LTS 2,LTS 4,LTS 3,LTS 4,LTS 4
7,ebike,scenario16,4.857143,5.500000,NaN,NaN,4.041667,NaN,NaN,5.000000,LTS 4,LTS 4,None,None,LTS 3,None,None,LTS 4
8,ebike,scenario17,NaN,NaN,NaN,NaN,4.347826,NaN,NaN,3.750000,None,None,None,None,LTS 3,None,None,LTS 3
9,ebike,scenario3,3.666667,4.018519,NaN,NaN,NaN,NaN,4.583333,3.952381,LTS 3,LTS 3,None,None,None,None,LTS 3,LTS 3


In [16]:
final_summary.to_csv('stress_class.csv', index=False)